In [1]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
import joblib

from discussion_agents.cog.functional.expel import (
    categorize_experiences,
    get_folds,
    create_rules,
)

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)

In [6]:
experiences = joblib.load("../../../../tests/assets/expel/expel_experiences_10_fake.joblib")

In [ ]:
from typing import Dict, Any

from discussion_agents.cog.modules.memory.base import BaseMemory


class ExpeLExperienceMemory(BaseMemory):
    def __init__(self, ) -> None:
        """Initialization."""
        super().__init__()

    def clear(self) -> None:
        pass

    def add_memories(self) -> None:
        pass

    def load_memories(self) -> Dict[str, Any]:
        pass

    def show_memories(self) -> Dict[str, Any]:
        pass